In [2]:
import requests
import pandas as pd
from glob import glob
from selenium import webdriver

In [3]:
category_save = "http://127.0.0.1:8081/main/category/save"
request_body = {"catename":"반팔"}
requests.post(url=category_save, json=request_body)

<Response [201]>

* 상의: 1
    * 반팔: 5
    * 긴팔: 6
    * 셔츠: 7

* 바지: 2
    * 반바지: 8
    * 슬랙스: 9
    * 데님팬츠: 10

* 스커트: 3
    * 미니스커트: 11
    * 롱스커트: 12

* 아우터: 4
    * 롱패딩: 13
    * 숏패딩: 14
    * 코트: 15
    * 트렌치 코트: 16

In [3]:
category_list = ['상의', '바지', '스커트', '아우터', '반팔', '긴팔', '셔츠', '반바지', '슬랙스', '데님팬츠', '미니스커트', '롱스커트', '롱패딩', '숏패딩', '코트', '트렌치 코트']

category_save = "http://127.0.0.1:8081/main/category/save"

for category in category_list:
    request_body = {"catename":category}
    requests.post(url=category_save, json=request_body)

In [4]:
csv_file = glob("./상품DataFrame/*.csv")
csv_file

['./상품DataFrame\\바지_데님팬츠.csv',
 './상품DataFrame\\바지_반바지.csv',
 './상품DataFrame\\바지_슬랙스.csv',
 './상품DataFrame\\상의_긴팔.csv',
 './상품DataFrame\\상의_반팔.csv',
 './상품DataFrame\\상의_셔츠.csv',
 './상품DataFrame\\스커트_롱스커트.csv',
 './상품DataFrame\\스커트_미니스커트.csv',
 './상품DataFrame\\아우터_롱패딩.csv',
 './상품DataFrame\\아우터_숏패딩.csv',
 './상품DataFrame\\아우터_코트.csv',
 './상품DataFrame\\아우터_트렌치 코트.csv']

In [4]:
tmp = csv_file[-1].split('\\')[1][:-4]
print(tmp.split('_'))

['아우터', '트렌치 코트']


In [5]:
tmp = csv_file[0].split('\\')[1][:-4]
print(tmp.split('_'))

['바지', '데님팬츠']


In [5]:
df = pd.read_csv(csv_file[0])
df = df.iloc[:, 1:]
df

,상품명,가격,이미지 경로,이미지 개수
0,CONA 9110 기모옵션 숏버전옵션 시크릿밴드 롱 루즈 와이드 데님 다크그레이,"45,000원",['./상품이미지/바지/데님팬츠/CONA 9110 기모옵션 숏버전옵션 시크릿밴드 롱...,3
1,CONA 9085 기모옵션추가 딥워싱 브러쉬 루즈핏 와이드 스트레이트 데님 진청,"46,800원",['./상품이미지/바지/데님팬츠/CONA 9085 기모옵션추가 딥워싱 브러쉬 루즈핏...,3
2,CONA 9154 기모옵션추가 커버밴드 와이드 루즈 데미지워싱 데님 블랙그레이,"48,600원",['./상품이미지/바지/데님팬츠/CONA 9154 기모옵션추가 커버밴드 와이드 루즈...,3
3,NOI024 스트레이트 루즈핏 데님 팬츠 블랙,"35,100원",['./상품이미지/바지/데님팬츠/NOI024 스트레이트 루즈핏 데님 팬츠 블랙1.j...,1
4,BUCKLE WIDE DENIM PANTS BLACK,"41,400원",['./상품이미지/바지/데님팬츠/BUCKLE WIDE DENIM PANTS BLAC...,3
...,...,...,...,...
85,W 카버 7801 하이웨이스트 세미와이드핏 다크인디고 데님팬츠,"59,500원",['./상품이미지/바지/데님팬츠/W 카버 7801 하이웨이스트 세미와이드핏 다크인디...,2
86,여성 슬림 데님_PHC1DP2022DIN,"69,900원",['./상품이미지/바지/데님팬츠/여성 슬림 데님_PHC1DP2022DIN1.jpg'],1
87,신디 M01 미드 블루,"129,000원","['./상품이미지/바지/데님팬츠/신디 M01 미드 블루1.jpg', './상품이미지...",3
88,로그 K11 라이트 블루,"129,000원","['./상품이미지/바지/데님팬츠/로그 K11 라이트 블루1.jpg', './상품이미...",3


In [7]:
df.columns

Index(['상품명', '가격', '이미지 경로', '이미지 개수'], dtype='object')

In [8]:
for i in range(5):
    name = df['상품명'][i]
    price = df['가격'][i]
    image_list = df['이미지 경로'][i][1:-1]
    for image in image_list.split(','):
        print(image.strip()[1:-1])

./상품이미지/바지/데님팬츠/CONA 9110 기모옵션 숏버전옵션 시크릿밴드 롱 루즈 와이드 데님 다크그레이1.jpg
./상품이미지/바지/데님팬츠/CONA 9110 기모옵션 숏버전옵션 시크릿밴드 롱 루즈 와이드 데님 다크그레이2.jpg
./상품이미지/바지/데님팬츠/CONA 9110 기모옵션 숏버전옵션 시크릿밴드 롱 루즈 와이드 데님 다크그레이3.jpg
./상품이미지/바지/데님팬츠/CONA 9085 기모옵션추가 딥워싱 브러쉬 루즈핏 와이드 스트레이트 데님 진청1.jpg
./상품이미지/바지/데님팬츠/CONA 9085 기모옵션추가 딥워싱 브러쉬 루즈핏 와이드 스트레이트 데님 진청2.jpg
./상품이미지/바지/데님팬츠/CONA 9085 기모옵션추가 딥워싱 브러쉬 루즈핏 와이드 스트레이트 데님 진청3.jpg
./상품이미지/바지/데님팬츠/CONA 9154 기모옵션추가 커버밴드 와이드 루즈 데미지워싱 데님 블랙그레이1.jpg
./상품이미지/바지/데님팬츠/CONA 9154 기모옵션추가 커버밴드 와이드 루즈 데미지워싱 데님 블랙그레이2.jpg
./상품이미지/바지/데님팬츠/CONA 9154 기모옵션추가 커버밴드 와이드 루즈 데미지워싱 데님 블랙그레이3.jpg
./상품이미지/바지/데님팬츠/NOI024 스트레이트 루즈핏 데님 팬츠 블랙1.jpg
./상품이미지/바지/데님팬츠/CONA 9213 커버밴드 보이핏 스트레이트 컷팅 데님 중청1.jpg
./상품이미지/바지/데님팬츠/CONA 9213 커버밴드 보이핏 스트레이트 컷팅 데님 중청2.jpg
./상품이미지/바지/데님팬츠/CONA 9213 커버밴드 보이핏 스트레이트 컷팅 데님 중청3.jpg


이미지 먼저 순회하면서 저장한 후 아이템 저장하고,
아이템에 해당 이미지들과 카테고리들을 넣어줌.

In [6]:
item_save_url = 'http://127.0.0.1:8081/main/item/save'
image_save_url = 'http://127.0.0.1:8081/main/image/save'
category_to_item = 'http://127.0.0.1:8081/main/category/add-to-item'
image_to_item = 'http://127.0.0.1:8081/main/image/add-to-item'

for csv in csv_file:
    tmp = csv.split('\\')[1][:-4]
    upper, lower = tmp.split('_') # 카테고리
    df = pd.read_csv(csv)
    df = df.iloc[:, 1:]

    for i in range(90):
        name = df['상품명'][i]
        price = df['가격'][i][:-1].split(',') # '원' 제거하고 , split
        price = int(''.join(price))
        image_list = df['이미지 경로'][i][1:-1] # [, ] 제거
        images = []
        for image_path in image_list.split(','):
            images.append(image_path.strip()[1:-1]) # 공백 제거 후 ', ' 제거

        # 아이템 저장
        item_request_body = {
            "itemname":name,
            "price":price,
            "purchasecnt":"0",
            "count":"200",
            "reviewmean":"0",
            "reviewcount":"0"}
        requests.post(url=item_save_url, json=item_request_body)

        # 이미지 저장
        for img in images:
            image_request_body = {
                'imgpath':img}
            requests.post(url=image_save_url, json=image_request_body)

        # Category To Item
        for cate in [upper, lower]:
            cate_item_body = {
                "itemName":name,
                "categoryName":cate}
            requests.post(url=category_to_item, json=cate_item_body)

        # Image to Item
        for img in images:
            image_request_body = {
                "itemName":name,
                "imgPath":img}
            requests.post(url=image_to_item, json=image_request_body)